In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [3]:
train_datagen_aug =  ImageDataGenerator(width_shift_range=0.15,
        height_shift_range=0.15, shear_range=0.15,
        validation_split=0.15,
        rescale=1./255)

test_datagen_aug =  ImageDataGenerator(width_shift_range=0.15,validation_split=0.5,
        height_shift_range=0.15, shear_range=0.15,
        zoom_range=0.15,
      
        rescale=1./255)


train_data = train_datagen_aug.flow_from_directory(r'D:\Uni-passau\thesis-smalldataset\dataset-50\train',target_size=(512,512), subset='training',
                                                       batch_size=50, class_mode='categorical')
valid_data = train_datagen_aug.flow_from_directory(r'D:\Uni-passau\thesis-smalldataset\dataset-50\train',target_size=(512,512),  subset='validation'
                                                       ,batch_size=20, class_mode='categorical')
#valid_data = test_datagen_aug.flow_from_directory(r'/content/drive/MyDrive/dataset-50/nor-dataset-50/test/',target_size=(256,256), 
                                                    #  batch_size=32, class_mode='categorical')

Found 3199 images belonging to 50 classes.
Found 550 images belonging to 50 classes.


In [ ]:
import keras.backend as k
import numpy as np
import random
seed = 12
random.seed(seed)
np.random.seed(seed)

In [4]:
from keras.regularizers import l2
from keras.layers import BatchNormalization
from keras.layers import GaussianNoise
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = (11,11), kernel_regularizer=l2(0.0001), padding = 'Same', activation ='relu', input_shape = (512,512,3)))
model.add(tf.keras.layers.Conv2D(filters = 48, kernel_size = (5,5), kernel_regularizer=l2(0.0001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides = 2))
model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),kernel_regularizer=l2(0.0001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.0001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.0001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation = "relu", kernel_regularizer=l2(0.0001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(512, activation = "relu", kernel_regularizer=l2(0.0001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation = "relu", kernel_regularizer=l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(50, activation = "softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 512, 512, 8)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 48)      9648      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 256, 48)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 128)     55424     
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 128)     512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 192)     2

In [3]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

csv = CSVLogger("/content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/50-model-augu_history_log(augu).csv", append=True)
es1= tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5, restore_best_weights=True)
mc = ModelCheckpoint('/content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/50-cnnmodel(augu).h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [5]:

adam = tf.keras.optimizers.Adam(learning_rate = 0.01, decay = 0.9)
model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [7]:
history = model.fit(train_data,epochs=300,shuffle=True,validation_data=valid_data,callbacks=[csv,es1,mc])

Epoch 1/300
64/64 [==============================] - 213s 3s/step - loss: 11.2916 - accuracy: 0.1013 - val_loss: 56.9573 - val_accuracy: 0.0200

Epoch 00001: val_accuracy improved from -inf to 0.02000, saving model to /content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/50-cnnmodel(augu).h5
Epoch 2/300
64/64 [==============================] - 224s 3s/step - loss: 11.0002 - accuracy: 0.1606 - val_loss: 15.5951 - val_accuracy: 0.0200

Epoch 00002: val_accuracy did not improve from 0.02000
Epoch 3/300
64/64 [==============================] - 226s 4s/step - loss: 10.7254 - accuracy: 0.1878 - val_loss: 11.9607 - val_accuracy: 0.0818

Epoch 00003: val_accuracy improved from 0.02000 to 0.08182, saving model to /content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/50-cnnmodel(augu).h5
Epoch 4/300
64/64 [==============================] - 226s 4s/step - loss: 10.3813 - accuracy: 0.2631 - val_loss: 10.9799 - val_accuracy: 0.1000

Epoch 00004: val_accuracy improved from 0.08182 to 0.10000

In [9]:
history = model.fit(train_data,epochs=300,shuffle=True,validation_data=valid_data,callbacks=[csv,es1,mc])

Epoch 1/300
64/64 [==============================] - 201s 3s/step - loss: 8.1060 - accuracy: 0.7872 - val_loss: 8.8636 - val_accuracy: 0.4745

Epoch 00001: val_accuracy did not improve from 0.89636
Epoch 2/300
64/64 [==============================] - 197s 3s/step - loss: 8.0588 - accuracy: 0.7962 - val_loss: 7.9178 - val_accuracy: 0.8873

Epoch 00002: val_accuracy did not improve from 0.89636
Epoch 3/300
64/64 [==============================] - 198s 3s/step - loss: 8.0012 - accuracy: 0.8144 - val_loss: 7.8682 - val_accuracy: 0.8945

Epoch 00003: val_accuracy did not improve from 0.89636
Epoch 4/300
64/64 [==============================] - 197s 3s/step - loss: 7.9812 - accuracy: 0.8134 - val_loss: 8.4694 - val_accuracy: 0.6000

Epoch 00004: val_accuracy did not improve from 0.89636
Epoch 5/300
64/64 [==============================] - 195s 3s/step - loss: 7.9741 - accuracy: 0.8047 - val_loss: 7.7768 - val_accuracy: 0.9000

Epoch 00005: val_accuracy improved from 0.89636 to 0.90000, savin

In [24]:
model.save_weights(r'/content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/cnn-augu-50_model-weights.h5')

In [6]:
model.load_weights(r'D:\Uni-passau\thesis-results\50-dataset\normal-cnn-augu\cnn-augu-50_model-weights.h5')

In [8]:
train_acc = model.evaluate(train_data)
val_acc = model.evaluate(valid_data)
print("train_Accuracy: %.2f%%" % (train_acc[1]*100))
print("valid_Accuracy: %.2f%%" % (val_acc[1]*100))

12/12 [==============================] - 156s 13s/step - loss: 7.3942 - accuracy: 0.8972
train_Accuracy: 93.18%
valid_Accuracy: 89.72%


In [7]:
val_acc = model.evaluate(valid_data)
print("valid_Accuracy: %.2f%%" % (val_acc[1]*100))

28/28 [==============================] - 108s 4s/step - loss: 7.2189 - accuracy: 0.9491
valid_Accuracy: 94.91%


In [8]:
test_datagen_nor = ImageDataGenerator(rescale=1./255)
test_datagen_aug =  ImageDataGenerator(width_shift_range=0.15,validation_split=0.5,
        height_shift_range=0.15, shear_range=0.15,
      
        rescale=1./255)

In [ ]:
## normal datset

In [ ]:
test_genrator_aug = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/nor-dataset-50/test',
        target_size=(256,256),
        batch_size=8,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
163/163 [==============================] - 32s 194ms/step - loss: 1.5635 - accuracy: 0.9331
Test_Accuracy: 93.31%


In [ ]:
## normal data augumentation

In [19]:
test_genrator_aug = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/nor-dataset-50/test',
        target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
17/17 [==============================] - 69s 4s/step - loss: 7.1764 - accuracy: 0.9477
Test_Accuracy: 94.77%


In [ ]:
## gaussian noise

In [21]:
test_genrator_gau_nor = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/gaussian-50/test',
        target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_gau_nor)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
17/17 [==============================] - 840s 52s/step - loss: 9.1659 - accuracy: 0.2392
Test_Accuracy: 23.92%


In [ ]:
## gaussian noise augumentation

In [22]:
test_genrator_gau_augmen = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/gaussian-50/test',
        target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_gau_augmen)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
17/17 [==============================] - 102s 6s/step - loss: 9.0516 - accuracy: 0.3223
Test_Accuracy: 32.23%


In [ ]:
## salt and pepper noise

In [8]:
test_genrator_salt = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/saltandpepper',
        target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_salt)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
17/17 [==============================] - 806s 49s/step - loss: 9.6809 - accuracy: 0.2015
Test_Accuracy: 20.15%


In [ ]:
## 20% corrupted resposes

In [ ]:
test_corrupted1= test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-20',
       target_size=(512,512),
        batch_size=60,
        class_mode='categorical')
test_acc = model.evaluate(test_corrupted1)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
 6/22 [=======>......................] - ETA: 3:34 - loss: 20.4078 - accuracy: 0.0361

In [ ]:
# 25 % corrupted

In [ ]:
test_genrator_corrupted = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-25\test'target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_corrupted)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

In [ ]:
## 50 corrupted resposes

In [ ]:
test_corrupted2= test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-50\test',
        target_size=(512,512),
        batch_size=80,
        class_mode='categorical')
test_acc = model.evaluate(test_corrupted2)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig('/content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/cnn-gaussain-accuracy.png')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig('/content/drive/MyDrive/dataset-50/nor-dataset-50/data-agu/cnn-gaussain-loss.png')
plt.show()

In [9]:
test_genrator_aug = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\dataset-50\test',
        target_size=(512,512),
        batch_size=20,
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
65/65 [==============================] - 258s 4s/step - loss: 7.3869 - accuracy: 0.9015
Test_Accuracy: 90.15%


In [11]:
test_genrator_nor = test_datagen_aug.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\dataset-50\test',
        target_size=(512,512),
        batch_size=50,
        class_mode='categorical')

Found 1300 images belonging to 50 classes.


In [12]:
x_test =[]
y_test =[]
len = int(1300/50)
print(len)
for i in range(len):
  x,y = next(test_genrator_nor)
  x_test.append(x)
  y_test.append(y)

26


In [13]:
import numpy as np
X_test =np.array(x_test)
Y_test = np.array(y_test)
print(Y_test.shape,X_test.shape)

(26, 50, 50) (26, 50, 512, 512, 3)


In [14]:
import numpy as np
n,batches,hei,wid,col=np.array(X_test).shape
X=np.array(X_test).reshape(n*batches,hei,wid,col)
x,y,z=np.array(Y_test).shape
Y =np.array(Y_test).reshape(x*y,z)
print(Y.shape,X.shape)

(1300, 50) (1300, 512, 512, 3)


In [ ]:
Y_pred = model.predict(X)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1)


In [ ]:
Y_classes = np.argmax(Y,axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_classes, Y_pred_classes))